# TimeAffineレイヤを確認する
TimeAffineレイヤは、Affineレイヤを時間方向に結合していくレイヤである

In [4]:
import numpy as np
from common.layers import Affine

In [5]:
class TimeAffine:
    def __init__(self, W, b):
        
        # パラメータのリスト
        self.params = [W, b]
        
        # 勾配のリスト
        self.grads = [np.zeros_like(W), np.zeros_like(b)]
        print(id(self.grads[0]))
        print(id(self.grads[1]))        
        self.x = None

    def forward(self, x):
        """
        順伝播計算
        x : 入力データ
        """
        N, T, D = x.shape # バッチサイズ、時間数、前層のノード数
        W, b = self.params

        # 全ての時刻について、一度でAffineの順伝播計算を行う
        rx = x.reshape(N*T, -1)
        out = np.dot(rx, W) + b # 行列の積 + バイアス
        
        # xを保持
        self.x = x
        
        return out.reshape(N, T, -1)

    def backward(self, dout):
        """
        逆伝播計算
        """
        x = self.x
        N, T, D = x.shape # バッチサイズ、時間数、前層のノード数
        W, b = self.params

        # 全ての時刻について、一度でAffineの逆伝播計算を行う
        dout = dout.reshape(N*T, -1)
        rx = x.reshape(N*T, -1)
        db = np.sum(dout, axis=0) # バイアスの勾配
        dW = np.dot(rx.T, dout) # 重みWの勾配
        dx = np.dot(dout, W.T) # 前層へ伝える勾配
        dx = dx.reshape(*x.shape)

        self.grads[0][...] = dW
        self.grads[1][...] = db
#         self.grads[0] = dW
#         self.grads[1] = db
    
        print(self.grads[0].shape, dW.shape)
        print(self.grads[1].shape, db.shape)
        
        
        print(id(self.grads[0]))
        print(id(self.grads[1]))
        
        return dx

In [6]:
np.random.seed(1234)
D = 1 # 入力層のノード数
H = 5 # 中間層のノード数
W = np.random.randn(D, H)
b = np.zeros(H)

# オブジェクトの生成
time_affine = TimeAffine(W, b)

# 順伝播計算
N = 4 # バッチサイズ
T = 5 # 時間数
x = np.random.randn(N, T, D)
out = time_affine.forward(x)
print("out=", out)
print()

# 逆伝播計算
dout = np.random.randn(N, T, H)
dx = time_affine.backward(dout)
print("dx=", dx)
print()


4493656672
4493657952
out= [[[ 4.18239806e-01 -1.05658950e+00  1.27104453e+00 -2.77373175e-01
   -6.39279619e-01]
  [ 4.05240205e-01 -1.02374891e+00  1.23153831e+00 -2.68751947e-01
   -6.19409726e-01]
  [-3.00079563e-01  7.58084023e-01 -9.11951660e-01  1.99010281e-01
    4.58671666e-01]
  [ 7.39982176e-03 -1.86939977e-02  2.24883017e-02 -4.90750050e-03
   -1.13106289e-02]
  [-1.05728055e+00  2.67098327e+00 -3.21311036e+00  7.01179703e-01
    1.61605351e+00]]

 [[ 5.42167280e-01 -1.36966460e+00  1.64766420e+00 -3.59560850e-01
   -8.28702786e-01]
  [ 4.67638235e-01 -1.18138360e+00  1.42116798e+00 -3.10133805e-01
   -7.14785128e-01]
  [ 4.49430516e-01 -1.13538587e+00  1.36583412e+00 -2.98058596e-01
   -6.86954626e-01]
  [-9.52890597e-01  2.40726536e+00 -2.89586587e+00  6.31949152e-01
    1.45649345e+00]
  [-1.57495818e-01  3.97878023e-01 -4.78634970e-01  1.04449922e-01
    2.40732386e-01]]

 [[ 9.98671601e-04 -2.52292085e-03  3.03499584e-03 -6.62310735e-04
   -1.52646972e-03]
  [ 1.911449